In [1]:
import torch
import numpy as np
import datasets
import evaluate
import mlflow
import os
import pandas as pd
from databricks import sql
from dotenv import load_dotenv
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
)
MODEL_CHECKPOINT = "facebook/mbart-large-50"


# Độ dài tối đa cho văn bản đầu vào và tóm tắt đầu ra
MAX_INPUT_LENGTH = 1024
MAX_TARGET_LENGTH = 512


mlflow.set_tracking_uri("http://172.17.0.1:5000")
experiment_name = "Summarization models"
mlflow.set_experiment(experiment_name)

<Experiment: artifact_location='s3://mlflow/3', creation_time=1759585443145, experiment_id='3', last_update_time=1759585443145, lifecycle_stage='active', name='Summarization models', tags={}>

In [2]:
database = "olist_dataset"
goldSche=database+ ".gold"
lookupGeo=database+".silver"+ ".silver_olist_geolocation"
lookupCateName=database+".silver"+ ".silver_product_category_name_translation"
connection = sql.connect(
                        server_hostname = "dbc-214ffec4-c1f2.cloud.databricks.com",
                        http_path = "/sql/1.0/warehouses/8753fa7395d762fe",
                        access_token = os.environ.get("DATABRICKS_TOKEN"))

In [3]:
query=f"""
SELECT * FROM {goldSche}.dim_olist_review_content
"""
df = pd.read_sql(query, connection)
df.describe()
df.info()

/tmp/ipykernel_2992/3806867786.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, connection)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98410 entries, 0 to 98409
Data columns (total 8 columns):
 #   Column                  Non-Null Count  Dtype              
---  ------                  --------------  -----              
 0   review_id               98410 non-null  object             
 1   order_id                98410 non-null  object             
 2   review_comment_title    11519 non-null  object             
 3   review_comment_message  40668 non-null  object             
 4   modifiedDate            98410 non-null  datetime64[ns, UTC]
 5   review_content_key      98410 non-null  int64              
 6   create_date             98410 non-null  datetime64[ns, UTC]
 7   update_date             98410 non-null  datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](3), int64(1), object(4)
memory usage: 6.0+ MB


In [4]:
# Sửa lại bằng cách thêm một cặp ngoặc vuông bên trong
df[['review_comment_title', 'review_comment_message']].head(20)

,review_comment_title,review_comment_message
0,None,"Produto veio como mostra na foto,veio certinho..."
1,None,PEDIDO FEITO A MAIS DE TRINTA DIAS E ATE AGORA...
2,None,None
3,Reclamação,Comprei dois itens. E só foi entregue um. Nota...
4,Recebi pedido incompleto,Paguei por 2 unidades do Perfume Miss Gabriela...
5,None,None
6,None,"Comprei 3 luminárias, e recebi apenas 1!\n"
7,None,None
8,None,produtos ok e dentro do prazo
9,None,None


In [12]:
from transformers import pipeline

device = 0 if torch.cuda.is_available() else -1
summarizer = pipeline(
    "summarization",
    model="sshleifer/distilbart-cnn-12-6",
    device=device      # Sử dụng GPU nếu có
)

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

In [13]:
import pandas as pd # Đảm bảo đã import pandas

# Lấy 5 bình luận ngẫu nhiên để thử nghiệm
sample_reviews = df['review_comment_message'].sample(50).tolist()

for i, review in enumerate(sample_reviews):
    # THÊM DÒNG KIỂM TRA NÀY VÀO ĐẦU VÒNG LẶP
    # Nó sẽ kiểm tra xem review có phải là None hoặc NaN không
    if not review or pd.isna(review):
        print(f"\n======== VÍ DỤ {i+1} ========")
        print("--> Bỏ qua vì bình luận bị rỗng (None/NaN).")
        continue

    # Code còn lại của bạn bây giờ sẽ an toàn vì nó chỉ chạy với các bình luận hợp lệ
    print(f"\n======== VÍ DỤ {i+1} ========")
    print(f"BÌNH LUẬN GỐC (độ dài: {len(review)} ký tự):")
    print(review)
    
    # Bỏ qua các bình luận quá ngắn
    if len(review.split()) < 8:
        print("\n--> Bỏ qua vì bình luận quá ngắn.")
        continue

    # Chạy inference
    summary = summarizer(
        review,
        max_length=100,
        min_length=10,
        do_sample=False
    )[0]['summary_text']
    
    print("\nBẢN TÓM TẮT (Zero-Shot):")
    print(summary)
    
print("\n==========================")
print("Hoàn thành!")

Your max_length is set to 100, but your input_length is only 49. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=24)



======== VÍ DỤ 1 ========
--> Bỏ qua vì bình luận bị rỗng (None/NaN).

======== VÍ DỤ 2 ========
BÌNH LUẬN GỐC (độ dài: 131 ký tự):
Excelente, ficou linda no meu Boris, super indico. Mercadoria de ótima qualidade, os produtos oram entregues antes do prazo marcado


Your max_length is set to 100, but your input_length is only 76. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=38)



BẢN TÓM TẮT (Zero-Shot):
 Mercadoria de ótima qualidade entregues antes do prazo marcado . Excelente, ficou linda no meu Boris, super indico.

======== VÍ DỤ 3 ========
--> Bỏ qua vì bình luận bị rỗng (None/NaN).

======== VÍ DỤ 4 ========
BÌNH LUẬN GỐC (độ dài: 194 ký tự):
Recebi o e-mail perguntando se eu recebi meu produto, porem já passou do prazo de entrega e ainda NÃO recebi meu produto, e diz que se eu não responder o e-mail seria dado como produto entregue.


Your max_length is set to 100, but your input_length is only 20. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=10)


...


BẢN TÓM TẮT (Zero-Shot):
 Produto foi entregue bementente antes do prazo . Chegou bem condicionado, excelente qualidade de atendimento .

======== VÍ DỤ 27 ========
BÌNH LUẬN GỐC (độ dài: 32 ký tự):
A ENTREGA FOI SUPER RÁPIDA AMEI 

--> Bỏ qua vì bình luận quá ngắn.

======== VÍ DỤ 28 ========
--> Bỏ qua vì bình luận bị rỗng (None/NaN).

======== VÍ DỤ 29 ========
--> Bỏ qua vì bình luận bị rỗng (None/NaN).

======== VÍ DỤ 30 ========
--> Bỏ qua vì bình luận bị rỗng (None/NaN).

======== VÍ DỤ 31 ========
--> Bỏ qua vì bình luận bị rỗng (None/NaN).

======== VÍ DỤ 32 ========
--> Bỏ qua vì bình luận bị rỗng (None/NaN).

======== VÍ DỤ 33 ========
--> Bỏ qua vì bình luận bị rỗng (None/NaN).

======== VÍ DỤ 34 ========
BÌNH LUẬN GỐC (độ dài: 27 ký tự):
Eu quero receber o produto.

--> Bỏ qua vì bình luận quá ngắn.

======== VÍ DỤ 35 ========
--> Bỏ qua vì bình luận bị rỗng (None/NaN).

======== VÍ DỤ 36 ========
--> Bỏ qua vì bình luận bị rỗng (None/NaN).

======== VÍ DỤ 37 ========
BÌN

Your max_length is set to 100, but your input_length is only 21. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=10)
Your max_length is set to 100, but your input_length is only 36. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=18)



BẢN TÓM TẮT (Zero-Shot):
 Amei o kit chegou com 8 dias q eu tinha pedido.. Muito bom , não faltou nenhum e veio sem nenhuma defeito .

======== VÍ DỤ 38 ========
--> Bỏ qua vì bình luận bị rỗng (None/NaN).

======== VÍ DỤ 39 ========
BÌNH LUẬN GỐC (độ dài: 49 ký tự):
Recebi um, a compra foram de 2, está faltando 01.

BẢN TÓM TẮT (Zero-Shot):
 Recebi um, acompra foram de 2, está faltando 01.01.

======== VÍ DỤ 40 ========
BÌNH LUẬN GỐC (độ dài: 31 ký tự):
Ponto positivo foi a entrega. 


--> Bỏ qua vì bình luận quá ngắn.

======== VÍ DỤ 41 ========
--> Bỏ qua vì bình luận bị rỗng (None/NaN).

======== VÍ DỤ 42 ========
--> Bỏ qua vì bình luận bị rỗng (None/NaN).

======== VÍ DỤ 43 ========
--> Bỏ qua vì bình luận bị rỗng (None/NaN).

======== VÍ DỤ 44 ========
--> Bỏ qua vì bình luận bị rỗng (None/NaN).

======== VÍ DỤ 45 ========
BÌNH LUẬN GỐC (độ dài: 96 ký tự):
O produto veio somente na caixa de envio, embalado no saco bolha, com aspecto de produto usado. 


You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
Your max_length is set to 100, but your input_length is only 20. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=10)



BẢN TÓM TẮT (Zero-Shot):
 O produto veio somente na caixa de envio, embalado no saco bolha, com aspecto de produto usado .

======== VÍ DỤ 46 ========
BÌNH LUẬN GỐC (độ dài: 60 ký tự):
Recebi o produto com nota fiscal diversa daquele que comprei

BẢN TÓM TẮT (Zero-Shot):
 Recebi o produto com nota fiscal diversa diversa daquele que comprei .

======== VÍ DỤ 47 ========
--> Bỏ qua vì bình luận bị rỗng (None/NaN).

======== VÍ DỤ 48 ========
--> Bỏ qua vì bình luận bị rỗng (None/NaN).

======== VÍ DỤ 49 ========
--> Bỏ qua vì bình luận bị rỗng (None/NaN).

======== VÍ DỤ 50 ========
--> Bỏ qua vì bình luận bị rỗng (None/NaN).

Hoàn thành!


In [15]:
MODEL_NAME="sshleifer/distilbart-cnn-12-6"
with mlflow.start_run(run_name="Zero-Shot Test with DistilBART") as run:
    run_id = run.info.run_id
    print(f"Bắt đầu MLflow Run ID: {run_id}")

    # Log tham số (ví dụ: tên model đã dùng)
    mlflow.log_param("model_name", MODEL_NAME)
    print(f"Đã log tham số: model_name = {MODEL_NAME}")

    # Log model (pipeline) vào MLflow
    # Chúng ta trích xuất model và tokenizer từ đối tượng pipeline
    print("Đang log model vào MLflow...")
    mlflow.transformers.log_model(
        transformers_model={
            "model": summarizer.model,
            "tokenizer": summarizer.tokenizer,
        },
        artifact_path="summarization_pipeline", # Tên thư mục lưu model trong run

    )
    print("Log model thành công!")

Bắt đầu MLflow Run ID: 7aa3cec528964097bf39be9bd4e68a18


2025/10/04 16:44:15 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


Đã log tham số: model_name = sshleifer/distilbart-cnn-12-6
Đang log model vào MLflow...


Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.
Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 142, 'min_length': 56, 'early_stopping': True, 'num_beams': 4, 'length_penalty': 2.0, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}
2025/10/04 16:44:17 WARNING mlflow.utils.requirements_utils: Found torch version (2.8.0a0+5228986c39.nv25.6) contains a local version label (+5228986c39.nv25.6). MLflow logged a pip requirement for this package as 'torch==2.8.0a0' without the local version label to make it installable from PyPI. To specify pip requirements containing local version 

Log model thành công!
🏃 View run Zero-Shot Test with DistilBART at: http://172.17.0.1:5000/#/experiments/3/runs/7aa3cec528964097bf39be9bd4e68a18
🧪 View experiment at: http://172.17.0.1:5000/#/experiments/3
